<a href="https://colab.research.google.com/github/Ignacio-Ibarra/Approximate-String-Matching/blob/main/Approximate_String_Matching_using_TF_IDF_over_Large_Dataset_Explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similitud entre Strings usando TF-IDF en Dataset de +11M de filas

## Introducción

El siguiente notebook es un trabajo realizado por el [CEP](https://www.argentina.gob.ar/produccion/cep) con el objetivo de encontrar una mejora las fuentes de información existentes aceca de las empresas que realizaron exportaciones a la Argentina, entre XXXX y XXXX. 

Se cuenta con una base de datos de más de 11M de filas con los nombres de compañias que han realizado ventas a la Argentina, pero cuyo nombre se encuentra por momentos mal escrito, escrito de varias formas distintas, con puntuaciones, distinos formatos, etc. El objetivo es reducir la cantidad de nombres distintos a la minima cantidad posible, limpiando los nombres sucios. 

Para ello, luego de la lectura bibliográfica correspondiente y el abordaje de distintas alternativas, se utilizó la técnica **TF-IDF** como método de vectorización y la distancia **_cosine similarity_** como forma de evaluación de la similitud entre los nombres de las empresas mediante. Previo a ello se realizó un preprocesamiento general de la base mediante el uso de **_regular expressions_**.  

## ¿Cómo se aborda este tipo de problemas?

Según bibliografía consultada uno de los método para comparar obtener similitud entre palabras o frases (en nuestro caso nombres de empresas) en datasets grandes es el metodo TF-IDF. 

**¿Por qué no la distancia de Levenshtein?**

La distancia de Levenshtein (también conocida como la distancia de edición) es uno de los métodos más usados para analizar la similitud entre dos strings. La similitud que analiza es qué tan parecidas son dos secuencias de caracteres y esta está medida en base a la cantidad de cambios que habría que hacerle al string A para que se convierta en string B. 

El problema que tenemos planteado es el de comparar cada uno de los nombres de las empresas que tenemos contra los otros N-1 otros nombres. Si bien es muy efectiva la medición de la similitud mediante esta distancia, tiene el problema de que no escala bien para datasets grandes, dado que el problema es de complejidad $\Omega(n^2)$. 

Es necesario encontrar otro método que permita resolver el problema de manera más eficiente para casos de _big data_. 

**¿Qué es TF-IDF?** 

TF-IDF (Term Frecuency-Invers Document Frecuency) es una técnica de consultas de información en documentos perteneciente al campo computacional conocido como [Information Retrieval](https://en.wikipedia.org/wiki/Information_retrieval). Este tipo de consulta es rankeada porque la búsqueda de documentos a partir de términos las realiza en base a la importancia que tiene un término en el documento y en base a la inversa de los documenos que contienen dicho documento. 

* Term Frecuency (TF): 

\begin{equation}
TF_{i,j} =\frac{Frecuencia Absoluta Termino_{i} Documento_{j}}{Cantidad Terminos Documento_{j}} = f_{i,j}
\end{equation}

* Inverse Document Frecuency (IDF): 

\begin{equation}
IDF_{i} =\log(\frac{N+1}{Cantidad Documentos Término_{i}})=\log(\frac{N+1}{d_{i}})
\end{equation}

* TF-IDF: 

\begin{equation}
TF-IDF_{i,j} =f_{i,j} .\log(\frac{N+1}{d_{i}})
\end{equation}

* Algunos plantean como opción para restarle importancia a TF: 

\begin{equation}
TF-IDF_{i,j} =\log(1+f_{i,j}).\log(\frac{N+1}{d_{i}})
\end{equation}

* Otra opcion es: 

\begin{equation}
TF-IDF_{i,j} =\log(1+\log(1+f_{i,j})).\log(\frac{N+1}{d_{i}})
\end{equation}

Considerando un documento que contiene 100 palabras en el cual la palabra gato aparece 3 veces. TF = (3/100) = 0.03. Ahora, asumiendo que en total hay 9.999.999 documentos y la palabra gato aparece en mil de esos documentos. IDF = log(9.999.999 + 1 / 1,000) = 4. Por lo tanto TF-IDF = 0.03*4 = 0.12. El número total de documentos fue elegido para que el logaritmo dé un nro redondo. 

Con la libería [scikit-learn](https://scikit-learn.org/stable/) se puede rapidamente calcular para cada termino de cada documento de toda una colección el respectivo valor TF-IDF. El cálculo que realiza el método `TfidfVectorizer` no es el estándar, ya que divide el valor por la norma del vector que está analizando. La matriz que se genera siempre en estos casos es de dimensión n x m siendo n la cantidad de documentos y m la cantidad de términos de toda la colección. Cada fila de la matriz estará compuesta por los respectivos valores tf-idf en aquellas columnas de los términos que se encuentren dentro del documento y zeros en todos las columnas de los términos que no están incluidos dentro del documento. Lo que arrojará una matriz esparsa (con muuuuchos ceros). 
Se puede decir que dos vectores poseen cierta similitud si ambos "apuntan en la misma dirección". 
De esta manera logramos traducir un vector de términos a un vector de números. Ejemplo, supongamos que tenemos tres documentos, cada uno con tres palabras.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
 
documentos = (
"La casa del perro está rota hace tiempo",
"El perro tiene la pata rota ahora",
"El perro no tiene la casa rota",
"El no perro perro perro")
 
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documentos)
print("Dimensiones de la Matriz: ", tfidf_matrix.shape)
print('**'*25)
print('Traducido a Matriz TF-IDF sería: \n')
words = tfidf_vectorizer.get_feature_names()
pd.DataFrame(tfidf_matrix.toarray(), columns=words)

Dimensiones de la Matriz:  (4, 13)
**************************************************
Traducido a Matriz TF-IDF sería: 



,ahora,casa,del,el,está,hace,la,no,pata,perro,rota,tiempo,tiene
0,0.000000,0.329977,0.418533,0.000000,0.418533,0.418533,0.267144,0.000000,0.000000,0.218408,0.267144,0.418533,0.000000
1,0.492895,0.000000,0.000000,0.314609,0.000000,0.000000,0.314609,0.000000,0.492895,0.257213,0.314609,0.000000,0.388604
2,0.000000,0.430157,0.000000,0.348249,0.000000,0.000000,0.348249,0.430157,0.000000,0.284716,0.348249,0.000000,0.430157
3,0.000000,0.000000,0.000000,0.342164,0.000000,0.000000,0.000000,0.422641,0.000000,0.839225,0.000000,0.000000,0.000000


**¿Como evaluamos ahora la similitud?**

Para definir la similitud entre dos documentos, para nuestro problema sería definir la similitud entre dos strings que indican nombres de empresas, es necesario introducir el concepto de producto escalar. Sean dos vectores $\vec{a} = (a_1, a_2, a_3, \ldots)$ y $\vec{b} = (b_1, b_2, b_3, \ldots)$, el producto escalar se define como:

\begin{equation}
\vec{a}\cdot\vec{b} = \sum_{n=1}^{n}a_{i}.b_{i}
\end{equation}

Desde un punto de visa geométrico el producto escalar es:

\begin{equation}
\vec{a} \cdot \vec{b} = \|\vec{a}\|\|\vec{b}\|\cos{\theta} 
\end{equation}

Acomodando la ecuación tenemos que: 

\begin{equation}
\vec{a} \cdot \vec{b} = \|\vec{b}\|\|\vec{a}\|\cos{\theta} 
\end{equation}

Siendo $\|\vec{a}\|\cos{\theta} $ la proyección del vector $\vec{a}$ en el vector $\vec{b}$. 

![](https://blog.christianperone.com/wp-content/uploads/2013/09/Dot_Product.png)

Cuanto más similares el ángulo $\theta$ es cada vez menor. Para valores de $\theta$ cercanos a $0^o$ el $\cos{\theta}$ tiende a $1$. Para valores cercanos a $90^o$ el valor de $\cos{\theta}$ tiende a $0$. Para valores cercanos a $180^o$ el valor tiende a $-1$ y para valores cercanos a $270^o$ tiende a $0$ nuevamente. Cuando el ángulo de dos vectores es igual a $90^o$ o $270^o$ se dice que los dos vectores son ortogonales. 

Por lo tanto para evaluar la similitud entre dos vectores tenemos que despejar la ecuacion: 

\begin{equation}
\vec{a} \cdot \vec{b} = \|\vec{a}\|\|\vec{b}\|\cos{\theta} \\ \\  \cos{\theta} = \frac{\vec{a} \cdot \vec{b}}{\|\vec{a}\|\|\vec{b}\|}
\end{equation}

Vectores muy opuestos deberían tener un _cosine similarity_ tendiendo a -1, vectores muy similaeres deberían tener un score tendiendo a 1.
 
![](https://blog.christianperone.com/wp-content/uploads/2013/09/cosinesimilarityfq1.png)



Para el ejemplo dado el coseno del ángulo entre el primer vector y el resto de los vectores es:

In [ ]:
from numpy import dot
from numpy.linalg import norm
 
def cos_between_vectors(a, b): 
  return dot(a, b.transpose())
 
 
a = tfidf_matrix.todense()[0,:]
b = tfidf_matrix.todense()[1,:]
c = tfidf_matrix.todense()[2,:]
d = tfidf_matrix.todense()[3,:]
 
N = len(tfidf_matrix.todense())
print('Cantidad de veces que hay que calcular el Cosine Similarity :', int((N*(N-1))/2))
 
 
a_b = cos_between_vectors(a,b)
print("\nCosine_Similarity entre vector a y b: ", a_b)
a_c = cos_between_vectors(a,c)
print("\nCosine_Similarity entre vector a y c: ", a_c)
a_d = cos_between_vectors(a,d)
print("\nCosine_Similarity entre vector a y d: ", a_d)
b_c = cos_between_vectors(b,c)
print("\nCosine_Similarity entre vector b y c: ", b_c)
b_d = cos_between_vectors(b,d)
print("\nCosine_Similarity entre vector b y d: ", b_d)
c_d = cos_between_vectors(c,d)
print("\nCosine_Similarity entre vector c y d: ", c_d)

Cantidad de veces que hay que calcular el Cosine Similarity : 6

Cosine_Similarity entre vector a y b:  [[0.22426947]]

Cosine_Similarity entre vector a y c:  [[0.39019161]]

Cosine_Similarity entre vector a y d:  [[0.18329353]]

Cosine_Similarity entre vector b y c:  [[0.56908022]]

Cosine_Similarity entre vector b y d:  [[0.32350765]]

Cosine_Similarity entre vector c y d:  [[0.53990119]]


Claramente, los vectores b y c son los más parecidos, aunque no en significado, pero sí en la composición de cada oración. Dado que en nuestro problema no tenemos que encontrar significados parecidos sino strings parecidas, es un método acertado para el problema. Aún así, tiene sus deficiencias este método porque, por ejemplo "INDUSTRIA COMERCIAL" será similar a "COMERCIAL INDUSTRIAL".  Una manera de resolver el orden de las palbras es separar las frases en _n-gramas_.

El otro problema es que los vectores c y d son parecidos. Esto ocurre porque como el valor TF-IDF en el documento d para la palabra "perro" es alto, porque se repite muchas veces en el mismo documento, entonces el valor de **TF** sube, por ende aumenta el producto escalar entre los vectores, lo que eleva la similitud. De todas maneras, cuando el corpus es grande, es decir el número de palabras distintas en la colección total de documentos es elevado, el peso que adquiere **TF** en el producto **TF x IDF** es menor. 

El benficio, como se comentó anteriormente es la eficiencia en realizar cálculos matriciales, lo que permite aumentar la capacidad de cómputo. 

**Uso de los N-Grams ¿Por qué?**

Para el análisis de similitud textual entre palabras, no referida al sentido, se puede hacer un análisis palabra a palabra o bien dividiendo las palabras en _n-gramas_. Estos son divisiones sucesivas de las palabras cada _n_ caracteres, por ejemplo: 

```
ngrams('MINISTERIO', n=2)

['MI', 'IN', 'NI', 'IS', 'ST', 'TE', 'ER', 'RI', 'IO']

ngrams('MINISTERIO', n=3)

['MIN', 'INI', 'NIS', 'IST', 'STE', 'TER', 'ERI', 'RIO']
```
Esto permite que ante problemas de mala escritura se puedan analizar de una misma palabra distintas partes de la misma. Esto permitiría una comparación más eficiente para el problema que tenemos en nuestro caso. 

**Eficiencia**

A dififerencia de los problemas de comparación string contra string, el caso de TF-IDF es más eficiente. En este caso para analizar la similitud entre cada vector simplemente tenemos que hacer la multiplicación de la matriz de valores de TF-IDF por su traspuesta. Y luego evaluar cada valor con la similitud del coseno. Incluso hay librerías que han realizado implementaciones aún más performante. 

* Uso de CSR Matrix: 

La matriz CSR (Compressed Sparse Row) es un método compuacional que comprime el uso de memoria guardando los datos que no son ceros de una matriz, mediante el uso de indices de fila, de columna y de valores. 
![](https://op2.github.io/PyOP2/_images/csr.svg)


* Selección de los strings más similares durante el proceso: 

El grupo de data scientists de la compañia ING, han implementado una optimización que permite obtener los strings más similares durante el proceso de computo guardando en memoria sólo las _n_ strings más similares([ver](https://medium.com/wbaa/https-medium-com-ingwbaa-boosting-selection-of-the-most-similar-entities-in-large-scale-datasets-450b3242e618)). 
